In [9]:
from muselsl import *
import time
import numpy as np
import serial
import matplotlib.pyplot as plt
from pylsl import StreamInlet, resolve_byprop
import warnings
from psycho_tracker import Calibrator, SHIFT_LENGTH, NUM_CHANNELS, EPOCH_LENGTH, OVERLAP_LENGTH, BUFFER_LENGTH, Band
import utils 
from process import ChannelDataProcessor
import xml.dom.minidom
#import data_processor
from metrics import *
from PsychoPy_Code.PsychoRun import run_psychopy

In [2]:
# Search for active LSL streams

print('Looking for an EEG stream...')
streams = resolve_byprop('type', 'EEG', timeout=2)
if len(streams) == 0:
    raise RuntimeError('Can\'t find EEG stream.')

# Set active EEG stream to inlet and apply time correction

print("Start acquiring data")
inlet = StreamInlet(streams[0], max_chunklen=12)
#eeg_time_correction = inlet.time_correction()

# Get the stream info and description
info = inlet.info()
#description = info.desc()




Looking for an EEG stream...
Start acquiring data


In [ ]:
"""
2. Obtain Threshold from PsychoPy 
"""

cal = Calibrator(inlet=inlet, info=info, channel_index=Channel.FP1)

cal.start_stage(mode='relax', stage=1)
time.sleep(1)

cal.end_stage(mode='relax', stage=1)

#run_psychopy(cal=cal)
#cal.update_stage_threshold()

In [62]:
"""
Estimate Relaxation from Band Powers
This example shows how to buffer, epoch, and transform EEG data from a single
electrode into values for each of the classic frequencies (e.g. alpha, beta, theta)
Furthermore, it shows how ratios of the band powers can be used to estimate
mental state for neurofeedback.
The neurofeedback protocols described here are inspired by
*Neurofeedback: A Comprehensive Review on System Design, Methodology and Clinical Applications* by Marzbani et. al
Adapted from https://github.com/NeuroTechX/bci-workshop
"""

# Handy little enum to make code more readable


""" 
1. CONNECT TO EEG STREAM 
"""
threshold = 0.0223716588909309062

print("FINAL THRESH", threshold)

"""
3. Record data and wave hand if metric is above threshold.  

"""
def get_fs(_info):
    return int(_info.nominal_srate())
fs = get_fs(info)
# The try/except structure allows to quit the while loop by aborting the
# script with <Ctrl-C>
print('Press Ctrl-C in the console to break the while loop.')


arduinoData = serial.Serial('com3',9600)
time.sleep(1)
try:
    # The following loop acquires data, computes band powers, and calculates neurofeedback metrics based on those band powers
    
#     c = data_processor.DataProcessor(buffer_length=BUFFER_LENGTH, epoch_length=EPOCH_LENGTH,
#                      overlap_length=OVERLAP_LENGTH, 
#                      shift_length=SHIFT_LENGTH, fs=fs, band_cls=Band, chans=utils.chans)
    
    while True:
        """ 3.1 ACQUIRE DATA """

        # Obtain EEG data from the LSL stream
        eeg_data, timestamp = utils.acquire_eeg_data(inlet,fs)           

        c = ChannelDataProcessor(buffer_length=BUFFER_LENGTH, epoch_length=EPOCH_LENGTH,
                      overlap_length=OVERLAP_LENGTH, 
                      shift_length=SHIFT_LENGTH, fs=fs, band_cls=Band)

        c.feed_new_data(eeg_data=eeg_data)  # Feed new data generated in the epoch
#        c.append_metrics()
#        print("-------")
#        print(c.df[""])

        metrics = [] # should be 4X8 matrix for each sample
        for i in range(NUM_CHANNELS):  # Iterate through all separate channels

            # Record channel smooth band power
            csbp = c.get_channel_smooth_band_powers(i) 
            # Run calculations on csbp to obtain desired metrics

            if i in [1,2,3]:
                theta_metric = csbp[1]

                metrics.append(theta_metric)
                #ratios.append(Metrics.beta_protocol(csbp, Band))
                print(theta_metric)
        metric = np.mean(metrics)  
        print(metric)    


        if check_positive(_metric=metric, _threshold=threshold): 

            hand_wave()
        else:
            hand_stay()

except KeyboardInterrupt:
    print('Closing!')   
arduinoData.close()

FINAL THRESH 0.022371658890930905
Press Ctrl-C in the console to break the while loop.
-0.004642733617260729
0.06448685301135283
-0.0029563088581979003
0.018962603511964733
hand stayed still
-0.0009178775232809128
0.06448690282487114
0.0029918198182391914
0.022186948373276474
hand stayed still
-0.0035970641386131306
0.06448601533263343
-0.03732523140481264
0.007854573263069218
hand stayed still
-0.011606140650861262
0.06448685301135283
-0.019981054982165576
0.010966552459441998
hand stayed still
-0.002408869477526289
0.06448685301135283
0.0021036059899899786
0.021393863174605506
hand stayed still
-0.004218332427239164
0.06447750702121893
-0.011862919906588644
0.016132084895797042
hand stayed still
-0.006777319646679764
0.06427816965872887
-0.007566776508002418
0.016644691168015566
hand stayed still
0.010776599325876404
0.06156441197900075
0.03623892757500809
0.03619331295996175
hand waved
-0.0389949213587069
0.05497382452694082
0.012055833474380738
0.009344912214204887
hand stayed stil

0.010279966836429953
0.0546568228646198
0.012179134995833054
0.025705308232294267
hand waved
-0.022916957917052196
0.06211455304660244
0.003356323173053596
0.01418463943420128
hand stayed still
-0.007208927356137536
0.06268278539761839
0.007626278363121456
0.021033378801534105
hand stayed still
-0.015323056166625628
0.060774984467522594
-0.005203871609516773
0.01341601889712673
hand stayed still
-0.012792015603844631
0.06013881230349744
0.012134978372682959
0.019827258357445258
hand stayed still
-0.019937658496678234
0.053036607511104936
-0.0039050472740988244
0.00973130058010929
hand stayed still
0.0031901301385721947
0.056021027768653975
0.03502434634859993
0.031411834751942036
hand waved
-0.01531454321529104
0.0558343106272133
0.016420961642123543
0.018980243018015267
hand stayed still
-0.01004075870025178
0.05374042927937363
-0.008080627631203314
0.011873014315972845
hand stayed still
-0.011825949605718184
0.05533634879520919
-0.012413574571779057
0.010365608205903981
hand stayed s

0.02161115067265062
0.026562127504738048
hand waved
-0.0006499618993194069
0.06455181629399045
0.012161355907078638
0.025354403433916563
hand waved
-0.00441839590599483
0.06303408054037572
0.01418538876261676
0.024267024465665882
hand waved
0.0012057116117795345
0.06879395842937594
0.0005608041014398365
0.02352015804753177
hand waved
0.005195641316407061
0.061195172531027985
-0.017695312928499487
0.016231833639645185
hand stayed still
-0.0016538522522547934
0.0605523312895068
-0.010473391425402342
0.016141695870616556
hand stayed still
-0.0022737616249603456
0.06793866000423388
-0.011755349717736221
0.017969849553845767
hand stayed still
-0.002627409856766574
0.04302517401353019
0.004763148857026832
0.015053637671263484
hand stayed still
-0.007065860488877702
0.06448685301135283
0.017219586930302892
0.024880193150926006
hand waved
-0.005833091770982799
0.06904533774894318
0.019393668521441913
0.0275353048331341
hand waved
0.0002436763621718277
0.049540965337202356
0.007354391053328408


-0.00487952369460574
0.007096803861851189
-0.02692511950658734
-0.008235946446447297
hand stayed still
-0.006807677112819559
0.027889115045807024
-0.038343930125489346
-0.005754164064167293
hand stayed still
0.0014967180543310342
0.04437846442676649
-0.013429373947143637
0.010815269511317963
hand stayed still
-0.016429700666675824
0.05542345446655033
0.01750694101295106
0.01883356493760852
hand stayed still
0.00795510550876195
0.061488201168923985
-0.017243626583316622
0.01739989336478977
hand stayed still
-0.005208243808237633
0.06448685301135283
0.009470415271286609
0.022916341491467273
hand waved
0.005270082439539391
0.06543769956932437
-0.045159817246736
0.008515988254042586
hand stayed still
-0.009467778789560889
0.03745996612952984
-0.00029275929545866396
0.00923314268150343
hand stayed still
-0.01032812325841085
0.04564418927101782
-0.011857778637447471
0.007819429125053167
hand stayed still
0.003231924721407229
0.011434809076313881
-0.021996058955180694
-0.0024431083858198616
h

0.003925465876867827
0.05184686445589669
-0.001512388750759046
0.018086647194001822
hand stayed still
-0.0077019349443244334
0.04755637803867226
-0.000959470862355742
0.012964990743997365
hand stayed still
-0.010034757857678782
0.05300867001757722
-0.008288984319984328
0.011561642613304703
hand stayed still
-0.006860783220738996
0.049843988570709985
-0.008782127766751816
0.01140035919440639
hand stayed still
-0.009002060272550853
0.04195614921880738
-0.012486937738427586
0.006822383735942979
hand stayed still
-0.005791961581490526
0.04334884846255234
-0.011350964584187807
0.008735307432291334
hand stayed still
0.00026302344096981706
0.05970447505443054
-0.018756600200084204
0.013736966098438719
hand stayed still
-0.007023777675429582
0.028694852788301417
-0.021909933689036026
-7.961952538806359e-05
hand stayed still
-0.011600213677666487
0.053376744569040936
-0.048657059193511945
-0.002293509434045831
hand stayed still
-0.0067301103869879435
0.05244877620709479
-0.010358030755044247
0.

0.020013446020702777
0.05717402070552624
-0.020291230502815227
0.018965412074471263
hand stayed still
0.017701609038047336
0.018225942705319177
-0.006515300239996688
0.009804083834456608
hand stayed still
0.018224092979528577
0.026908638074119574
0.0035181178672075647
0.01621694964028524
hand stayed still
0.01283389223466742
0.038360885741443265
-0.0031336203579460462
0.01602038587272155
hand stayed still
0.006014123107084264
0.03271582739018612
-0.014817060638493326
0.00797096328625902
hand stayed still
-0.011025686423435686
0.026504539309659427
0.026034750898259157
0.013837867928160967
hand stayed still
-0.027241316819283427
0.04420843412920444
-0.0011630977888364932
0.005268006507028172
hand stayed still
-0.022922598283954217
0.05706247846987857
-0.028459504268459276
0.001893458639155027
hand stayed still
-0.026925900934472464
0.0687435350634477
-0.00278016492781549
0.013012489733719915
hand stayed still
-0.026238378291491216
0.02959146811681233
-0.012465175095340671
-0.003037361756

-0.028268844971187465
0.05646336482130257
-0.0217747563821492
0.0021399211559886347
hand stayed still
0.02090429797701114
0.07054927385348106
0.06423515869733205
0.05189624350927475
hand waved
0.016598449917799372
0.07338522611938021
0.05939081789874782
0.04979149797864246
hand waved
0.012951587788135863
0.0591352777057245
0.0269277616008648
0.033004875698241724
hand waved
0.004985965567146752
0.05147594750222887
0.028564516653046927
0.028342143240807516
hand waved
0.021874196482267412
0.07099114157439287
0.03467509944219795
0.04251347916628608
hand waved
0.016481994256220513
0.07522595045028596
0.009961309218667521
0.03388975130839133
hand waved
0.010299468628446573
0.05936404841064565
0.005012690197395424
0.024892069078829213
hand waved
0.007670707603143175
0.025771811997791497
-0.024135981649179555
0.0031021793172517063
hand stayed still
-0.004356749368843416
0.06254426786364545
0.027732525777306255
0.028640014757369426
hand waved
-0.0018836607211259225
0.06294863790732091
-0.008048

0.06817460255841816
0.0706304167293677
0.044494218710037475
0.06109974599927445
hand waved
0.07234020153645865
0.06565476674228564
0.05081813960324383
0.06293770262732938
hand waved
0.07677512674638061
0.07521322551588268
0.05208221465634793
0.06802352230620375
hand waved
0.06057953896053198
0.06110870982153499
0.04241365849057895
0.05470063575754864
hand waved
0.06424101945104536
0.06503973605716322
0.041308324656975294
0.05686302672172796
hand waved
0.06831630818904022
0.06503973605716322
0.0535299134307199
0.06229531922564111
hand waved
0.03911812391451441
0.06503973605716322
0.047210802772516616
0.05045622091473142
hand waved
0.05929933655132765
0.06503973605716322
0.038108318104060616
0.05414913023751716
hand waved
0.027480980622354732
0.06503973605716322
0.036438268031166926
0.04298632823689496
hand waved
0.04025810016545802
0.06733757553734884
0.04995828552175598
0.05251798707485428
hand waved
0.04023811450534951
0.06503973605716322
0.04842752798413994
0.05123512618221756
hand w

0.06213518047333105
0.0645915041076375
0.05790537665610965
0.0615440204123594
hand waved
0.04018494685963556
0.04559656555510401
0.05331898498008024
0.046366832464939944
hand waved
0.042317668411248587
0.044764729710948394
0.049142541354389696
0.04540831315886223
hand waved
0.04999079803881781
0.04650372531858825
0.04671516080863985
0.047736561388681965
hand waved
0.03943118612687215
0.04626676927502603
0.037903336601871895
0.04120043066792336
hand waved
0.04552005671908325
0.04672364991312187
0.048303112342390694
0.046848939658198606
hand waved
0.04171800338105708
0.06636788954244456
0.03774633480407671
0.04861074257585945
hand waved
0.06503973605716322
0.06383198093173859
0.03594946290088726
0.05494039329659636
hand waved
0.07038503400023566
0.07189771291516561
0.039081365372652895
0.06045470409601805
hand waved
0.058362293346068295
0.06503973605716322
0.04157122931001854
0.054991086237750016
hand waved
0.055776715220695404
0.046338921327253646
0.043071886088071325
0.0483958408786734

In [64]:
arduinoData.close()

In [51]:
def check_positive(_metric, _threshold):
    """
    Returns True if beta_metric is less than the threshold 
    :return: 
    """
    return _metric > _threshold

def hand_wave():
    print("hand waved")
    action = '1'    #pass in the input of either 1 or 0
    arduinoData.write(action.encode())

def hand_stay():
    print("hand stayed still")
    action = '0'
    arduinoData.write(action.encode())